# Lab B.4 Solutions: Evaluation and Analysis

Solutions to exercises from Lab B.4.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 1 Solution: NDCG with Graded Relevance

In [ ]:
def ndcg_graded(ranked_items, relevance_scores, k=10):
    """
    NDCG with graded (non-binary) relevance.
    
    Args:
        ranked_items: List of item IDs in ranked order
        relevance_scores: Dict mapping item_id -> relevance score (e.g., rating)
        k: Cutoff position
    
    Returns:
        NDCG value between 0 and 1
    
    Example:
        >>> ranked = [1, 2, 3, 4, 5]
        >>> relevance = {1: 5, 2: 4, 3: 3, 5: 5}  # Item 4 has no relevance
        >>> ndcg = ndcg_graded(ranked, relevance, k=5)
    """
    # Get relevances for ranked items
    relevances = []
    for item in ranked_items[:k]:
        relevances.append(relevance_scores.get(item, 0.0))
    
    relevances = np.array(relevances)
    
    # DCG: sum of relevance / log2(position + 1)
    positions = np.arange(1, len(relevances) + 1)
    discounts = np.log2(positions + 1)
    dcg = np.sum(relevances / discounts)
    
    # Ideal DCG: best possible ranking
    ideal_relevances = sorted(relevance_scores.values(), reverse=True)[:k]
    if len(ideal_relevances) < k:
        ideal_relevances = list(ideal_relevances) + [0.0] * (k - len(ideal_relevances))
    
    ideal_positions = np.arange(1, len(ideal_relevances) + 1)
    ideal_discounts = np.log2(ideal_positions + 1)
    idcg = np.sum(np.array(ideal_relevances) / ideal_discounts)
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg


# Example with MovieLens ratings
# Simulated: user rated items 10, 20, 30 with ratings 5, 3, 4
relevance = {10: 5, 20: 3, 30: 4}

# Perfect ranking (by relevance)
perfect_ranking = [10, 30, 20, 1, 2, 3, 4, 5, 6, 7]  # 10 (5), 30 (4), 20 (3), rest 0
ndcg_perfect = ndcg_graded(perfect_ranking, relevance, k=10)

# Suboptimal ranking
suboptimal_ranking = [20, 30, 10, 1, 2, 3, 4, 5, 6, 7]  # 20 (3), 30 (4), 10 (5) - reversed
ndcg_suboptimal = ndcg_graded(suboptimal_ranking, relevance, k=10)

# Bad ranking (relevant items at bottom)
bad_ranking = [1, 2, 3, 4, 5, 6, 7, 10, 20, 30]
ndcg_bad = ndcg_graded(bad_ranking, relevance, k=10)

print("NDCG with Graded Relevance:")
print(f"  Perfect ranking:    NDCG = {ndcg_perfect:.4f}")
print(f"  Suboptimal ranking: NDCG = {ndcg_suboptimal:.4f}")
print(f"  Bad ranking:        NDCG = {ndcg_bad:.4f}")

print("\n📊 Graded relevance penalizes putting 5-star items lower than 3-star items!")

---

## Exercise 2 Solution: MMR Re-ranking

In [ ]:
def mmr_rerank(item_scores, item_embeddings, lambda_param=0.5, k=10):
    """
    Maximal Marginal Relevance re-ranking for diversity.
    
    MMR = argmax [lambda * Rel(d,q) - (1-lambda) * max_s Sim(d,s)]
    
    Args:
        item_scores: Dict mapping item_id -> relevance score
        item_embeddings: Dict mapping item_id -> embedding vector
        lambda_param: Trade-off between relevance (1) and diversity (0)
        k: Number of items to select
    
    Returns:
        List of selected item_ids
    """
    selected = []
    candidates = list(item_scores.keys())
    
    for _ in range(k):
        if not candidates:
            break
        
        best_item = None
        best_mmr = -float('inf')
        
        for item in candidates:
            # Relevance term
            relevance = item_scores[item]
            
            # Diversity term: max similarity to already selected
            if selected:
                similarities = [
                    cosine_similarity(item_embeddings[item], item_embeddings[s])
                    for s in selected
                ]
                max_sim = max(similarities)
            else:
                max_sim = 0
            
            # MMR score
            mmr = lambda_param * relevance - (1 - lambda_param) * max_sim
            
            if mmr > best_mmr:
                best_mmr = mmr
                best_item = item
        
        selected.append(best_item)
        candidates.remove(best_item)
    
    return selected


def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8)


# Example
np.random.seed(42)

# Create some items with embeddings
# Group 1: Action movies (similar embeddings)
# Group 2: Comedies (similar embeddings)
# Group 3: Documentaries (similar embeddings)

item_embeddings = {}
item_scores = {}

# Action movies (items 0-4)
action_base = np.random.randn(16)
for i in range(5):
    item_embeddings[i] = action_base + np.random.randn(16) * 0.1
    item_scores[i] = 0.9 - i * 0.02  # High scores

# Comedies (items 5-9)
comedy_base = np.random.randn(16)
for i in range(5, 10):
    item_embeddings[i] = comedy_base + np.random.randn(16) * 0.1
    item_scores[i] = 0.85 - (i-5) * 0.02  # Medium-high scores

# Documentaries (items 10-14)
doc_base = np.random.randn(16)
for i in range(10, 15):
    item_embeddings[i] = doc_base + np.random.randn(16) * 0.1
    item_scores[i] = 0.7 - (i-10) * 0.02  # Medium scores

# Pure relevance ranking
relevance_ranking = sorted(item_scores.keys(), key=lambda x: -item_scores[x])[:10]

# MMR re-ranking with different lambdas
mmr_high_relevance = mmr_rerank(item_scores, item_embeddings, lambda_param=0.9, k=10)
mmr_balanced = mmr_rerank(item_scores, item_embeddings, lambda_param=0.5, k=10)
mmr_high_diversity = mmr_rerank(item_scores, item_embeddings, lambda_param=0.2, k=10)

print("Comparison of Rankings:")
print(f"  Pure relevance:    {relevance_ranking}")
print(f"  MMR (λ=0.9):       {mmr_high_relevance}")
print(f"  MMR (λ=0.5):       {mmr_balanced}")
print(f"  MMR (λ=0.2):       {mmr_high_diversity}")

# Count genres in each
def count_genres(ranking):
    action = sum(1 for r in ranking if r < 5)
    comedy = sum(1 for r in ranking if 5 <= r < 10)
    docs = sum(1 for r in ranking if r >= 10)
    return f"Action:{action}, Comedy:{comedy}, Docs:{docs}"

print("\nGenre Distribution:")
print(f"  Pure relevance: {count_genres(relevance_ranking)}")
print(f"  MMR (λ=0.5):    {count_genres(mmr_balanced)}")
print(f"  MMR (λ=0.2):    {count_genres(mmr_high_diversity)}")

In [ ]:
# Visualize the trade-off
lambdas = np.arange(0, 1.05, 0.1)
diversities = []
avg_relevances = []

for lam in lambdas:
    ranking = mmr_rerank(item_scores, item_embeddings, lambda_param=lam, k=10)
    
    # Measure diversity (number of unique genres)
    genres = set()
    for item in ranking:
        if item < 5:
            genres.add('action')
        elif item < 10:
            genres.add('comedy')
        else:
            genres.add('docs')
    diversities.append(len(genres))
    
    # Average relevance
    avg_rel = np.mean([item_scores[i] for i in ranking])
    avg_relevances.append(avg_rel)

fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.plot(lambdas, avg_relevances, 'b-o', label='Avg Relevance')
ax1.set_xlabel('Lambda (λ)')
ax1.set_ylabel('Average Relevance', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.plot(lambdas, diversities, 'r-s', label='Genre Diversity')
ax2.set_ylabel('Number of Genres', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.title('MMR Trade-off: Relevance vs Diversity')
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 Observations:")
print("   - λ=1: Pure relevance, low diversity (all action movies)")
print("   - λ=0.5: Balanced - good relevance AND diversity")
print("   - λ=0: Pure diversity, lower relevance")

---

## Key Takeaways

1. **Graded NDCG**: Uses actual ratings instead of binary relevance. Better for explicit feedback.

2. **MMR**: Balances relevance and diversity with λ parameter. λ=0.5-0.7 is typical.

3. **Trade-offs**: There's always a trade-off between accuracy and diversity. Choose based on business goals.

4. **Evaluation**: Use multiple metrics. No single metric captures everything!